# Les fonctions avancées en PowerShell

## C'est quoi une fonction ?

<font size="4"> Une fonction c'est une série d'instructions qui s'exécute quand on appelle la fonction </font>

```powershell
Function Test {
    Write-Host "Hello World"
}
```

## C'est quoi la différence avec un script ? 

### Script

<font size="4"> Pour lancer le script vous devez soit mettre le chemin complet du script soit vous déplacer dans le dossier ou se trouve le script.. </font>

In [ ]:
$PWD

In [ ]:
./01-ScriptFile.ps1

In [ ]:
Set-Location -Path ./Demo
./01-ScriptFile.ps1

In [ ]:
Set-Location -Path ..
$PWD

In [ ]:
./Demo/01-ScriptFile.ps1

### Fonction

<font size="4"> Il faut voir la fonction comme un CmdLet personnel. 

Une fois la fonction chargée en mémoire, elle est disponible tant que vous ne fermez pas votre instance de PowerShell. </font>

In [ ]:
Function DoSomeThing {
    Write-host "Je suis la fonction"
}

In [ ]:
DoSomeThing

<font size="4"> Je peux me déplacer dans n'importe quel dossier ma fonction sera toujours disponible puisqu'elle est chargée en mémoire. </font>

## Les bonnes pratiques

<font size="4"> Voici quelques Best Practice que j'essai de m'appliquer à moi-même :
* Ne compliquer pas les choses. Faites au plus simple et direct
* N'utiliser pas d'alias dans votre fonction. (on oublie les ipmo, ii ...  )
* Metre en forme le code pour en simplifier la relecture
* Ne coder pas de valeur en dur mais utiliser des paramètres
* Utiliser au maximum des noms de paramètres standards
* Pour nommer votre fonction utiliser un verb approuvé et un nom singulier
</font>

In [ ]:
Get-Verb | Select-Object Verb,Group,Description | Sort-Object Group,Verb

In [ ]:
(Get-Command -ParameterName "ComputerName" -ErrorAction SilentlyContinue).count

In [ ]:
(Get-Command -ParameterName "Computer" -ErrorAction SilentlyContinue).count

## Fonction

L'exemple ci-dessous présente une fonction "basique"

In [ ]:
Function Test-Parameter {
    param(
        $ComputerName
    )

    Write-Output "ComputerName = $ComputerName"
}

Test-Parameter -ComputerName "MONORDI"

Si on vérifie les paramètre de cette fonction on voit qu'elle n'a qu'un seul paramètre

In [ ]:
Get-Command -Name Test-Parameter -Syntax

Si on vérifie la liste des noms de paramètre disponible :

In [ ]:
(Get-Command -Name Test-Parameter).Parameters.Keys

Mais il nous manque les paramètres habituels qu'on retrouve sur quasiment tous les CmdLet PowerShell tel que verbose ou debug.
C'est pour cela qu'on passe en fonction avancée.

## Fonction avancée

Le passage en fonction avancée est assez simple il suffit d'ajouter ```[CmdletBinding()]``` a notre fonction ainsi que les blocs Begin/Process/End.

Les blocs ne sont pas obligatoire pour faire une fonction avancée

In [ ]:
Function Test-Parameter {
    [CmdletBinding()]
    param(
        [System.String[]]$ComputerName
    )

    Begin {}
    Process {
        Write-Output "ComputerName = $ComputerName"
    }
    End {}
   
}

Test-Parameter -ComputerName "MONORDI"

Si on regarde la syntax maintenant on voit un ```[<CommonParameters>]``` apparaitre

In [ ]:
Get-Command -Name Test-Parameter -Syntax

En regardant la liste des noms de paramètres on retrouve tous les paramètres habituels

In [ ]:
(Get-Command -Name Test-Parameter).Parameters.Keys

Les blocs Begin et End ne sont executés qu'une seule fois alors que le bloc Process sera executé autant de fois qu'il y a de valeur passé en paramètre 

In [ ]:
Function Test-Parameter {
    [CmdletBinding()]
    param(
        [System.String[]]$ComputerName
    )

    Begin {
        Write-Verbose "Je suis le bloc Begin"
    }
    Process {
        foreach ($Name in $ComputerName) {
            Write-Verbose "ComputerName = $Name"
        }
    }
    End {
        Write-verbose "Je suis le bloc End"
    }
   
}

Test-Parameter -ComputerName "MONORDI1","MONORDI2","MONORDI3","MONORDI4" -Verbose

## Validation des paramètres

### Mandatory

Le souci dans l'exemple précédent est que l'on peut lancer la fonction sans lui passer de paramètre.

Pour corriger cela on va rendre le pramètre obligatoire.

In [ ]:
Function Test-Parameter {
    [CmdletBinding()]
    param(
        [Parameter(Mandatory)]
        [System.String[]]$ComputerName
    )

    Write-Output "ComputerName = $ComputerName"
}

Test-Parameter -ComputerName "MONORDI"
Test-Parameter

### [ValidateNotNullOrEmpty()] et [ValidateNotNull()]

Va nous permettre de nous assurer qu'un valeur est bien passé a notre paramètre.

```ValidateNotNull``` : vérifie uniquement si la valeur transmise au paramètre est une valeur null. Cela fonctionnera toujours s’il est passé une chaîne vide.

In [ ]:
Function Test-ParameterNotNull {
    [CmdletBinding()]
    param(
        [ValidateNotNull()]
        [System.String[]]$ComputerName
    )

    Write-Output "ComputerName = $ComputerName"
}

Test-ParameterNotNull -ComputerName $null 
Test-ParameterNotNull -ComputerName ""

```ValidateNotNullOrEmpty``` : vérifie également si la valeur transmise est une valeur null et s’il s’agit d’une chaîne ou d’une collection vide

In [ ]:
Function Test-ParameterNotNullOrEmpty {
    [CmdletBinding()]
    param(
        [ValidateNotNullOrEmpty()]
        [System.String[]]$ComputerName
    )

    Write-Output "ComputerName = $ComputerName"
}

Test-ParameterNotNullOrEmpty -ComputerName $null 
Test-ParameterNotNullOrEmpty -ComputerName ""

### [ValidateLength()]

Cette validation permet de définir une taille minimale et maximale attendue pour un paramètre.

Dans notre exemple nous voulons que le paramètre ```ComputerName``` fasse minimum 1 caractère et au maximum 13 caractères.

In [ ]:
Function Test-ParameterLength {
    [CmdletBinding()]
    param(
        [ValidateLength(1,13)]
        [System.String[]]$ComputerName
    )

    Write-Output "ComputerName = $ComputerName"
}

Test-ParameterLength -ComputerName "MONORDI"
Test-ParameterLength -ComputerName "MONNOMORDITROPLONG"


### [ValidateRange()]

C’est en quelque sorte le pendant de ```[ValidateLength()]``` mais pour les chiffres. 

Comme pour ```[ValidateLength()]```, il permet de définir une valeur minimale et maximale que peut prendre un paramètre.

Dans notre exemple l’age du PC doit-être compris entre 5 et 10 ans

In [ ]:
Function Test-ParameterRange {
    [CmdletBinding()]
    param(
        [ValidateLength(1,13)]
        [System.String[]]$ComputerName,
        [ValidateRange(5,10)]
        [System.Int32]$Age
    )

    Write-Output "ComputerName = $ComputerName"
    Write-Output "Age = $Age"
}

Test-ParameterRange -ComputerName "MONORDI" -Age 15
Test-ParameterRange -ComputerName "MONORDI" -Age 7

### [ValidateCount()]

Comme pour ValidateRange() il permet de définir le nombre minimal et le nombre maximal d’objet que peux prendre une collection passée en paramètre.

Dans notre exemple on peut passer plusieurs nom d'ordinateur à la fois, mais pas plus de 3.

In [ ]:
Function Test-ParameterCount {
    [CmdletBinding()]
    param(
        [ValidateCount(1,3)]
        [System.String[]]$ComputerName,
        [ValidateRange(5,10)]
        [System.Int32]$Age
    )

    Write-Output "ComputerName = $ComputerName"
    Write-Output "Age = $Age"
}

test-ParameterCount -ComputerName "MONORDI1","MONORDI2","MONORDI3","MONORDI4" -Age 6
test-ParameterCount -ComputerName "MONORDI1","MONORDI2","MONORDI3" -Age 6

### [ValidateSet()]

Cette validation permet de fournir une liste de réponse possible pour la valeur du paramètre.

Cette validation peut-être rendu Case Sensitive, si à la fin de la déclaration, la valeur ignorecase est définie à $False

Dans cette exemple les valeurs du paramètre ```site``` ne sont pas case sensitive.

In [ ]:
Function Test-ParameterSet {
    [CmdletBinding()]
    param(
        [ValidateCount(1,3)]
        [System.String[]]$ComputerName,
        [ValidateRange(5,10)]
        [System.Int32]$Age,
        [ValidateSet("SIEGE","AGENCE","STOCK")]
        [System.String]$Site
    )

    Write-Output "ComputerName = $ComputerName"
    Write-Output "Age = $Age"
    Write-Output "Site = $Site"
}

Test-ParameterSet -ComputerName "MONORDI1" -Age 6 -Site HOME
Test-ParameterSet -ComputerName "MONORDI1" -Age 6 -Site siege
Test-ParameterSet -ComputerName "MONORDI1" -Age 6 -Site SIEGE

Dans cette exemple les valeurs du paramètre ```site``` sont case sensitive.

In [ ]:
Function Test-ParameterSet {
    [CmdletBinding()]
    param(
        [ValidateCount(1,3)]
        [System.String[]]$ComputerName,
        [ValidateRange(5,10)]
        [System.Int32]$Age,
        [ValidateSet("SIEGE","AGENCE","STOCK",ignorecase=$false)]
        [System.String]$Site
    )

    Write-Output "ComputerName = $ComputerName"
    Write-Output "Age = $Age"
    Write-Output "Site = $Site"
}

Test-ParameterSet -ComputerName "MONORDI1" -Age 6 -Site HOME
Test-ParameterSet -ComputerName "MONORDI1" -Age 6 -Site siege
Test-ParameterSet -ComputerName "MONORDI1" -Age 6 -Site SIEGE

### [ValidatePattern()]

Permet d’utiliser une expression régulière (REGEX) avant de tester la valeur d’un paramètre

Dans notre exemple nous voulons être sur que le paramètre ComputerName soit toujours sous la forme d’un string commençant par ```SRV-``` suivi de 6 lettres et de 3 chiffres.

In [ ]:
Function Test-ParameterPattern {
    [CmdletBinding()]
    param(
        [ValidatePattern('^SRV-[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]\d{3}$')]
        [System.String[]]$ComputerName,
        [ValidateRange(5,10)]
        [System.Int32]$Age,
        [ValidateSet("SIEGE","AGENCE","STOCK")]
        [System.String]$Site
    )

    Write-Output "ComputerName = $ComputerName"
    Write-Output "Age = $Age"
    Write-Output "Site = $Site"
}

Test-ParameterPattern -ComputerName "MONORDI" -Age 6 -Site SIEGE
Test-ParameterPattern -ComputerName "SRV-ABCDEF001" -Age 6 -Site SIEGE

### [ValidateScript()]

Cette validation permet de définir un script qui va être executer pour valider la valeur du paramètre.

Imaginons que nous voulions ajouter un paramètre ```Path``` qui serait le dossier vers lequel nous allons enregister notre liste d'ordinateur.

Nous voulons nous assurer que ce chemin est bien un dossier.

In [ ]:
Function Test-ParameterScript {
    [CmdletBinding()]
    param(
        [ValidatePattern('^SRV-[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]\d{3}$')]
        [System.String[]]$ComputerName,
        [ValidateRange(5,10)]
        [System.Int32]$Age,
        [ValidateSet("SIEGE","AGENCE","STOCK")]
        [System.String]$Site,
        [ValidateScript(
            {IF (!(Test-Path -Path $_ -PathType Container)) {
                Throw "$($Path) n'est pas un dossier valide"
            } else {
                $true
            }
            }
        )]
        [System.String]$Path
    )

    Write-Output "ComputerName = $ComputerName"
    Write-Output "Age = $Age"
    Write-Output "Site = $Site"
    Write-Output "Path = $Path"
}

Test-ParameterScript -ComputerName "SRV-ABCDEF001","SRV-ABCDEF002","SRV-ABCDEF003" -Age 6 -Site SIEGE -Path c:\MONPATH
Test-ParameterScript -ComputerName "SRV-ABCDEF001","SRV-ABCDEF002","SRV-ABCDEF003" -Age 6 -Site SIEGE -Path C:\Temp

Comme vous pouvez l’imaginer, le script peut-être quelque chose de plus compliqué qu’un simple test comme ici.

### Utiliser plusieurs validations

Dans l'exemple ci-dessous, on voit que la paramètre ```ComputerName``` a plusieurs validation.

In [ ]:
Function Test-ParameterCumul {
    [CmdletBinding()]
    param(
        [ValidatePattern('^SRV-[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]\d{3}$')] #Le nom du serveur est de la forme SRV-ABCDEF001
        [ValidateCount(1,3)] #On ne peut passer que 1, 2 ou 3 valeurs a ce paramètre
        [ValidateLength(1,13)] #le nom ne doit pas dépasser 13 caractères
        [ValidateNotNullOrEmpty()] #La valeur ne doit-être ni null et ni vide
        [System.String[]]$ComputerName,
        [ValidateRange(5,10)]
        [System.Int32]$Age,
        [ValidateSet("SIEGE","AGENCE","STOCK")]
        [System.String]$Site,
        [ValidateScript(
            {IF (!(Test-Path -Path $_ -PathType Container)) {
                Throw "$($Path) n'est pas un dossier valide"
            } else {
                $true
            }
            }
        )]
        [System.String]$Path
    )

    Write-Output "ComputerName = $ComputerName"
    Write-Output "Age = $Age"
    Write-Output "Site = $Site"
}

Test-ParameterCumul -ComputerName "SRV-ABCDEF001","SRV-ABCDEF002","SRV-ABCDEF003" -Age 6 -Site SIEGE -Path C:\Temp
Test-ParameterCumul -ComputerName "SRV-ABCDEF001","MONORDI","SRV-ABCDEF003" -Age 6 -Site SIEGE -Path C:\Temp

Pour ceux qui ont suivis, la validation ```ValidateLength(1,13)``` ne sert pas grand chose ici puisqu'il y a déjà la valdiation ```[ValidatePattern('^SRV-[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]\d{3}$')]```

## Les jeux de paramètres

### 1 Paramètre dans chaque jeu de paramètres

Les jeux de paramètres permettent de définir un ensemble de paramètre qui doivent être utiliser par une fonction.

Cela permet par exemple de définir pour une même fonction 2 paramètres différents en entrée. 

Si l'on reprend notre exemple, on peut imaginer une fonction qui prendrait en paramètre soit un nom d'ordinateur soit un fichier avec une liste de nom d'ordinateur.

In [ ]:
Function Test-ParameterSetName {
    [CmdletBinding()]
    param(
        [System.String[]]$ComputerName
    )

    Foreach($Computer in $ComputerName) {
        Write-Output "ComputerName = $Computer"
    }
}

Test-ParameterSetName -ComputerName "SRV-ABCDEF001","SRV-ABCDEF002","SRV-ABCDEF003"

In [ ]:
Function Test-ParameterSetName {
    [CmdletBinding()]
    param(
        [System.IO.FileInfo]$Path
    )

    $ComputerName = (Import-Csv -Path $Path -Delimiter ",").computername  

    Foreach($Computer in $ComputerName) {
        Write-Output "ComputerName = $Computer"
    }
}

test-ParameterSetName -Path ".\Demo\ComputerList.csv"

On se rend compte que finalement c'est 2 fonctions font quasiment la même chose.

On va donc utilisé le ```ParameterSetName``` pour définir le jeu de paramètre et ainsi faire 1 fonction qui prendra l'un ou l'autre des paramètre (Nom ou Fichier)

In [ ]:
Function Test-ParameterSetName {
    [CmdletBinding(DefaultParameterSetName="ByName")]    #Je définis le nom du jeu de paramètre par défaut
    param(
        [Parameter(ParameterSetName="ByName")] # Je définis le nom du jeu de paramètre pour mon paramètre ComputerName
        [System.String[]]$ComputerName,
        [Parameter(ParameterSetName="ByFile")] # Je définis le nom du jeu de paramètre pour mon paramètre Path
        [System.IO.FileInfo]$Path
    )

    switch ($PSCmdlet.ParameterSetName) {    # Je vérifis le jeu de paramètre utilisé
        "ByName" {
            Write-Output "ByName"
            $ComputerName = $ComputerName
        }
        "ByFile" {
            Write-Output "ByFile"
            $ComputerName = (Import-Csv -Path $Path -Delimiter ",").computername
        }
    }

    Foreach($Computer in $ComputerName) {
        Write-Output "ComputerName = $Computer"
    }
}

Test-ParameterSetName -ComputerName "SRV-ABCDEF001","SRV-ABCDEF002","SRV-ABCDEF003"
Test-ParameterSetName -Path ".\Demo\ComputerList.csv"

In [ ]:
Get-Command -Name Test-ParameterSetName -Syntax

En vérifiant la syntaxe de ma fonction j'ai bien 2 syntaxes différentes qui sont exclusive l'une de l'autre.

Je ne peux pas appeller cette fonction avec le paramètre CompterName et avec le paramètre Path en même temps

In [ ]:
Test-ParameterSetName -ComputerName "SRV-ABCDEF001","SRV-ABCDEF002","SRV-ABCDEF003" -Path ".\Demo\ComputerList.csv"

### 1 Paramètre dans plusieurs jeux de paramètre

Dans les exemples précédents nous avions un paramètre ```Age``` qui correspond à l'age des ordinateurs.

Ce paramètre doit-être utilisable dans les 2 jeux de paramètres ```ByName``` et ```ByFile```

In [ ]:
Function Test-ParameterSetName {
    [CmdletBinding(DefaultParameterSetName="ByName")]    #Je définis le nom du jeu de paramètre par défaut
    param(
        [Parameter(ParameterSetName="ByName")] # Je définis le nom du jeu de paramètre pour mon paramètre ComputerName
        [System.String[]]$ComputerName,
        [Parameter(ParameterSetName="ByFile")] # Je définis le nom du jeu de paramètre pour mon paramètre Path
        [System.IO.FileInfo]$Path,
        [Parameter(ParameterSetName="ByFile")]
        [Parameter(ParameterSetName="ByName")]
        [System.Int32]$Age
    )

    switch ($PSCmdlet.ParameterSetName) {    # Je vérifis le jeu de paramètre utilisé
        "ByName" {
            Write-Output "ByName"
            $ComputerName = $ComputerName
        }
        "ByFile" {
            Write-Output "ByFile"
            $ComputerName = (Import-Csv -Path $Path -Delimiter ",").computername
        }
    }

    Foreach($Computer in $ComputerName) {
        Write-Output "ComputerName = $Computer"
        Write-Output "Age = $Age"
    }
}

En vérifiant la syntaxe, cette fois on retrouve bien notre paramètre ```Age``` dans les 2 jeux de paramètres.

In [ ]:
Get-Command -Name Test-ParameterSetName -Syntax


Test-ParameterSetName [-ComputerName <string[]>] [-Age <int>] [<CommonParameters>]

Test-ParameterSetName [-Path <FileInfo>] [-Age <int>] [<CommonParameters>]



In [ ]:
Test-ParameterSetName -ComputerName "SRV-ABCDEF001","SRV-ABCDEF002","SRV-ABCDEF003" -Age 5
Test-ParameterSetName -Path ".\Demo\ComputerList.csv" -Age 7

## Les paramètres dynamiques

Dans l'exemple sur ```ValidateSet``` nous avons vu que nous pouvions passer une liste de valeurs en paramètre.

In [ ]:
Function Test-ParameterSet {
    [CmdletBinding()]
    param(
        [ValidateSet("MyFile1","MyFile2","MyFile3")]
        [System.String]$FileName
    )

    Write-Output "FileName = $FileName"
}

In [ ]:
Test-ParameterSet -FileName MyFile2

C'est déjà super cool, mais ce serait encore plus cool si la liste des fichier du paramètre ```FileName``` pouvait être dynamique, en prenant par exemple la liste des fichiers dans un sous-dossier.

La différence entre un paramètre standard et un paramètre dynamqique est que le paramètre dynammique est dans son propre bloc de commande.

```powershell
[CmdletBinding()]
param()
DynamicParam {

}
```


Pour faire simple, un paramètre dynamique est un object ```System.Management.Automation.RuntimeDefinedParameterDictionary``` dans lequel on trouve un ou plusieurs objects de type ```System.Management.Automation.RuntimeDefinedParameter``` 

Mais ce n'est pas aussi simple que ca en fait ;-) 

On va essayer de décortiquer tout ca.

In [ ]:
# Etape 1 : la création de l'object System.Management.Automation.RuntimeDefinedParameterDictionary
$RuntimeParamDic = New-Object System.Management.Automation.RuntimeDefinedParameterDictionary

In [ ]:
# Etape 2 : la création d'un object System.Collections.ObjectModel.Collection qui va contenir des objects System.Attribute
$AttribColl = New-Object System.Collections.ObjectModel.Collection[System.Attribute]

In [ ]:
#Etape 3 : creation d'un object System.Management.Automation.ParameterAttribute dans lequel on défini tous les attributs que notre paramètre va pouvoir avoir
# par exemple : qu'il soit utilisable dans tous les jeux de paramètre de ma fonction, qu'il soit obligatorie (Mandatory) ???
$ParamAttrib = New-Object System.Management.Automation.ParameterAttribute
$ParamAttrib.Mandatory = $Mandatory.IsPresent
$ParamAttrib.ParameterSetName = '__AllParameterSets'

In [ ]:
# Etape 4 : on ajoute notre ParameterAttribute a notre collection d'attribut créé précedement
$AttribColl.Add($ParamAttrib)

In [ ]:
# Etape 5 : comme je veux utiliser ValidateSet pour la validation de mon paramètre je dois l'ajouter à notre collection d'attribut
# c'est également a cette endroit que je défini le code qui va créer les valeurs que pourra prendre mon paramètre dynamique
$AttribColl.Add((New-Object System.Management.Automation.ValidateSetAttribute((Get-ChildItem .\Demo\FileName -File | Select-Object -ExpandProperty Name))))

In [ ]:
# Etape 6 : je vais définir mon paramètre dynamique, son type et ces attributs
$RuntimeParam = New-Object System.Management.Automation.RuntimeDefinedParameter('FileName', [string], $AttribColl)

In [ ]:
# Etape 7 : Ajouter ce RuntimeParam a mon object créé à l'étape 1
$RuntimeParamDic.Add('FileName', $RuntimeParam)

In [ ]:
# Etape 8 : Pour finir on retourne notre object System.Management.Automation.RuntimeDefinedParameterDictionary
return $RuntimeParamDic

In [ ]:
Function Test-ParameterDynamic {
    [CmdletBinding()]
    param(
    )
    DynamicParam {
        $RuntimeParamDic = New-Object System.Management.Automation.RuntimeDefinedParameterDictionary
        $AttribColl = New-Object System.Collections.ObjectModel.Collection[System.Attribute]
        $ParamAttrib = New-Object System.Management.Automation.ParameterAttribute
        $ParamAttrib.Mandatory = $Mandatory.IsPresent
        $ParamAttrib.ParameterSetName = '__AllParameterSets'
        $AttribColl.Add($ParamAttrib)
        $AttribColl.Add((New-Object System.Management.Automation.ValidateSetAttribute((Get-ChildItem .\Demo\FileName -File | Select-Object -ExpandProperty Name))))
        $RuntimeParam = New-Object System.Management.Automation.RuntimeDefinedParameter('FileName', [string], $AttribColl)
        $RuntimeParamDic.Add('FileName', $RuntimeParam)
        return $RuntimeParamDic
    }

    begin {

    }

    Process {
    }

    End {
        
    }
}

In [ ]:
Test-ParameterDynamic -FileName 